# Fiche Pratique - Bag of Words & TF-IDF


La technique de vectorisation d'un document textuel appelée "Bag of Words" est très efficace pour mener des analyses qui reposent sur l'exploitation des **champs lexicaux**. Grâce à cette technique de preprocessing, les modèles auront un accès privilégié à la fréquence d'apparition des mots (mais perdront toute information sur le sens dans lequel les mots apparaissent : les négations par exemple sont mal encodées, ce qui rend la technique plus adaptée à la classfication du sujet d'un document qu'à la détermination de son contenu positif ou négatif par exemple).

**Excellente référence** à laquelle se reporter pour plus de détails : [Scikit-learn User Guide](https://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction)

## O. Quelques définitions préliminaires

- **tokenizing** : processus de division d'un texte en plusieurs éléments appelés tokens. Par exemple, en utilisant les espaces et la ponctuation comme séparateurs
- **counting** : compter les occurences de chaque token au sein d'un document
- **normalizing** : processus de réduction de l'importance des tokens qui apparaissent dans la majorité des documents (ou samples), souvent en utilisant des poids décroissants

## 1. Sacs de mots

### 1.1 Approche classique
Le "Bag of Words" (sac de mots) est une représentation des textes où chaque mot d'un document est réduit à un token, sans tenir compte de son ordre ou de sa grammaire, mais uniquement de sa fréquence.


In [7]:
from sklearn.feature_extraction.text import CountVectorizer

# Exemple de documents
documents = ["le chat dort", "le chien dort"]

# Création du modèle Bag of Words
vectorizer = CountVectorizer()

# Application du modèle aux documents
X = vectorizer.fit_transform(documents)

print(vectorizer.get_feature_names_out())
print(X.toarray())

['chat' 'chien' 'dort' 'le']
[[1 0 1 1]
 [0 1 1 1]]


In [4]:
# Dans le cas d'un nouveau mot
print(vectorizer.transform(["Le lapin dort."]).toarray())
# Le Lapin n'est évidemment pas reconnu mais la majuscule et le point sont
# correctement ignorés

[[0 0 1 1]]


#### Remarque :
- On peut bien entendu pondérer les comptages par la taille du document pour obtenir des fréquences. On voit alors apparaître un problème (ou une feature un peu contre intuitive) : pour les longs documents, seuls les termes vraiment très communs tels que "et", "ou" etc. se verront affublés d'un fort coefficient.

### 1.2 Bi-grams
On peut également construire le vocabulaire sur des ensemble de plusieurs mots, ce qui permet de mieux appréhender les négations mais qui augmente considérablement la taille du dictionnaire (heureusement que scikit-learn utilise des matrices sparses :)

In [8]:
from os import XATTR_CREATE
from sklearn.feature_extraction.text import CountVectorizer

# Exemple de documents
documents = ["le chat dort", "le chien dort"]

# Création du modèle Bag of Words avec bigrams
vectorizer = CountVectorizer(ngram_range=(2, 2), token_pattern=r'\b\w+\b')

# Application du modèle aux documents
X = vectorizer.fit_transform(documents)

print(vectorizer.get_feature_names_out())
print(X.toarray())

['chat dort' 'chien dort' 'le chat' 'le chien']
[[1 0 1 0]
 [0 1 0 1]]


#### Remarque :
- On voit ici l'expression regulière utilisée par scikit-learn avec le caractère `\b` pour word boundary, ce qui permet de savoir exactement les retraitements effectués

## 2. TF - IDF (Term Frequency - Inverse Document Frequency)
Le TD-IDF se réfère à une technique de normalisation qui permet de minimiser l'importance des termes présents dans la plupart des documents du corpus. Contrairement au Bag of Words, cette technique n'a donc de sens que sur un **corpus de documents** et non sur un document unique.

*Remarque* : Si on travaille avec un unique article Wikipedia par exemple, il est possible de considérer que l'on dispose tout de même d'un corpus de phrases... ;)

### 2.1 Définition

La coordonnée correspondant à un token $t \in \{1, 2, 3, \ldots, n\}$ au sein d'un document $d$ est donnée par la formule :

$$\text{tf-idf(t,d)}=\frac{\text{tf(t,d)} \times \text{idf(t)}}{\sqrt{\text{tf-idf(1,d)}^2 + \ldots + \text{tf-idf(n,d)}^2}}$$

où $\text{tf(t,d)}$ correspond au nombre d'occurences du token $t$ dans le document $d$ (comme défini précédemment) et

$$\text{idf}(t) = \log{\frac{1 + n}{1+\text{df}(t)}} + 1$$

avec $n$ le nombre total de documents dans le corpus et $\text{df}(t)$ le nombre de documents du corpus qui contiennent le token $t$.

Cette définition de l'IDF affecte les tokens qui apparaissent d'un coefficient égal à 1 et d'un coefficient supérieur à 1 pour les autres.

### 2.2 Exemple et comparaison avec et sans pondération

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

documents = [
    "le chat dort",
    "le chien dort",
    "le chat et le chien dorment",
    "le lapin joue"
]

# Avec pondération
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(documents)

pd.DataFrame(columns=vectorizer.get_feature_names_out(), data=X.toarray())

,chat,chien,dorment,dort,et,joue,lapin,le
0,0.640434,0.000000,0.000000,0.640434,0.000000,0.000000,0.000000,0.423897
1,0.000000,0.640434,0.000000,0.640434,0.000000,0.000000,0.000000,0.423897
2,0.378779,0.378779,0.480433,0.000000,0.480433,0.000000,0.000000,0.501420
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.663385,0.663385,0.346182


In [14]:
# Sans pondération : le document n°2 semble mal représenté...
vectorizer = TfidfVectorizer(use_idf=False)
X = vectorizer.fit_transform(documents)

pd.DataFrame(columns=vectorizer.get_feature_names_out(), data=X.toarray())

,chat,chien,dorment,dort,et,joue,lapin,le
0,0.577350,0.000000,0.000000,0.57735,0.000000,0.00000,0.00000,0.577350
1,0.000000,0.577350,0.000000,0.57735,0.000000,0.00000,0.00000,0.577350
2,0.353553,0.353553,0.353553,0.00000,0.353553,0.00000,0.00000,0.707107
3,0.000000,0.000000,0.000000,0.00000,0.000000,0.57735,0.57735,0.577350
